In [ ]:
import pandas as pd

dati_presenze = pd.read_excel("....xlsx")
#dati_presenze.shape

dati_presenze

In [ ]:
from collections import defaultdict as dd
import datetime

#dd_presenze = dd(list)
dd_presenze = dd(dict)


for i  in dati_presenze.index:
    #print(dati_presenze["Data"][i], dati_presenze["T1"][i])
    """
    dd_presenze[dati_presenze["Data"][i]].append({dati_presenze["ID"][i] : [dati_presenze["T1"][i], dati_presenze["T2"][i], dati_presenze["T3"][i], dati_presenze["T4"][i], \
                                                                            dati_presenze["T5"][i], dati_presenze["T6"][i], dati_presenze["T7"][i], dati_presenze["T8"][i], \
                                                                            dati_presenze["T9"][i], dati_presenze["T10"][i], dati_presenze["T11"][i], dati_presenze["T12"][i]]})
    

    dd_presenze[dati_presenze["Data"][i]].update({dati_presenze["ID"][i] : [dati_presenze["T1"][i], dati_presenze["T2"][i], dati_presenze["T3"][i], dati_presenze["T4"][i], \
                                                                            dati_presenze["T5"][i], dati_presenze["T6"][i], dati_presenze["T7"][i], dati_presenze["T8"][i], \
                                                                            dati_presenze["T9"][i], dati_presenze["T10"][i], dati_presenze["T11"][i], dati_presenze["T12"][i]]})
    """
    dd_presenze[dati_presenze["Data"][i]].update({dati_presenze["ID"][i] : [dati_presenze["T"+str(ind)][i] for ind in range(1,13) if dati_presenze["T"+str(ind)][i] != datetime.time(0,0)]})

for k, v in dd_presenze.items():
    print(len(v), k, v)



In [ ]:
from datetime import datetime, time, timedelta

"""
def to_timestamp(value: datetime.time) -> str:
    dt = time(hour = value.hour, minute = value.minute)
    dt = dt.strftime("%H:%M:%S") #https://docs.python.org/3/library/time.html#time.strftime
    return dt
"""
def convert_to_q(orario):
    ora, minuti = orario.hour, orario.minute
    quarto_dora = ((ora * 60 + minuti) // 15)+1
    return quarto_dora


def quarter_range(start, end):
    quarters = []
    current = datetime.combine(datetime.min, start) #converto gli orari in oggetti datetime prima di eseguire l'operazione +=
    end = datetime.combine(datetime.min, end)

    while current <= end:
        q = convert_to_q(current)
        quarters.append(q)

        current += timedelta(minutes=15)

    return quarters

#def define_ranges()
dd_intervalli = dd(list)
for data, info in dd_presenze.items():
    for dip, orari in info.items():
        #print(orari)
        orari_ingresso = orari[::2]
        orari_uscita = orari[1::2]
        #print(orari_ingresso, orari_uscita)
        intervalli=[]
        for i in range(len(orari_ingresso)):
            try:
                print(orari_ingresso[i],  orari_uscita[i])
                intervalli += quarter_range(orari_ingresso[i], orari_uscita[i])#pd.date_range(orari_ingresso[i], orari_uscita[i], freq='15T').time
                #print(intervallo)
            except IndexError:
                print(f"Per l'ID {dip} in data {data.date} non risulta il timbro di uscita dopo l'ingresso delle {orari_ingresso[i]}")
        dd_intervalli[data].append(intervalli)

print(dd_intervalli)


In [132]:
import numpy as np

def create_q_matrix(dct_intervalli):
    matrix = np.zeros((len(dct_intervalli), 96))
    for i, (date, ranges) in enumerate(dct_intervalli.items()):
        for _range in ranges:
            for quarter in _range:
                matrix[i, quarter-1] +=1
        #matrix[i, ]
        """
        q = [0 for i in range(1,97)]
        for _range in ranges:
            for quarter in _range:
                q[quarter] += 1
        all_q.append(q)
        """

    return matrix

matrice_presenze = create_q_matrix(dd_intervalli)
matrice_presenze



array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [128]:
date = dati_presenze["Data"].unique()
q_ora_lab = [f"Q{str(i)}" for i in range(1,97)]

#date, q_ora

df_presenze = pd.DataFrame(data=matrice_presenze)
df_presenze.columns = q_ora_lab
df_presenze["Data"] = date
df_presenze = df_presenze.set_index("Data")
df_presenze.to_csv("prova_presenze.csv")


In [ ]:
import seaborn as sns

p = sns.lineplot(data=df_presenze.T)
p.set_title("Presenze Rettorato per Quarti d'Ora")
p.set_xlabel("Tempo (15 min)")
p.set_ylabel("num. presenze")